In [1]:
import mysql.connector #type:ignore
import json

In [14]:
path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\data\output\dump_iti_16_40select.json"
with open(path, "r", encoding="utf-8") as f:
    data = json.load(f)

print(data.keys())

dict_keys(['iti multi cap fund', 'iti elss tax saver fund', 'iti large cap fund', 'iti mid cap fund', 'iti small cap fund', 'iti value fund', 'iti pharma and healthcare fund', 'iti banking and financial services fund', 'iti flexi cap fund', 'iti focused equity fund', 'iti large & mid cap fund', 'iti balanced advantage fund', 'iti arbitrage fund', 'iti overnight fund', 'iti liquid fund', 'iti ultra short duration fund', 'iti banking & psu debt fund', 'iti dynamic bond fund'])


In [15]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="1234",
    database="this_db"
)
cursor = conn.cursor()

def get_or_create_amc(amc_name):
    cursor.execute("SELECT id FROM amcs WHERE amc_name = %s", (amc_name,))
    result = cursor.fetchone()
    if result:
        return result[0]
    cursor.execute("INSERT INTO amcs (amc_name) VALUES (%s)", (amc_name,))
    conn.commit()
    return cursor.lastrowid


In [16]:
for scheme_name, details in data.items():
    amc_id = get_or_create_amc(details["amc_name"])
    
    # mutual fund data
    mutual_fund_keys = ["amc_id","amc_name","benchmark_index", "main_scheme_name", "mutual_fund_name", "monthly_aaum_date", "monthly_aaum_value", "scheme_launch_date", "min_addl_amt", "min_addl_amt_multiple", "min_amt", "min_amt_multiple"]
    mutual_fund_query = f"INSERT INTO mutual_funds ({', '.join(mutual_fund_keys)}) VALUES ({', '.join(['%s'] * len(mutual_fund_keys))});"
    values = [amc_id] + [details[key] for key in mutual_fund_keys[1:]]
    cursor.execute(mutual_fund_query, values)
    
    mutual_fund_id = cursor.lastrowid
    
    # fund manager data
    fund_manager_keys = ["amc_id", "mutual_fund_id", "main_scheme_name", "name", "qualification", "managing_fund_since", "total_exp"]
    fund_manager_query = f"INSERT INTO fund_managers ({', '.join(fund_manager_keys)}) VALUES ({', '.join(['%s'] * len(fund_manager_keys))});"

    for manager in details.get("fund_manager", []):
        values = [amc_id, mutual_fund_id, details["main_scheme_name"]] + [manager.get(col, '') for col in fund_manager_keys[3:]]
        cursor.execute(fund_manager_query, values)
        
    # load data
    load_keys = ['amc_id', 'mutual_fund_id', 'main_scheme_name', 'entry_load', 'exit_load']
    load_query = f"INSERT INTO transformed_loads ({', '.join(load_keys)}) VALUES ({', '.join(['%s'] * len(load_keys))});"
    values = [amc_id, mutual_fund_id, details["main_scheme_name"]] + [details.get("load", {}).get(col, '') for col in load_keys[3:]]
    cursor.execute(load_query, values)
    
    # metrics
    metric_keys = ['amc_id', 'main_scheme_name', 'mutual_fund_id', 'alpha', 'avg_div_yld', 'avg_maturity', 'avg_pb', 'avg_pe', 'beta', 'jenson', 'macaulay_duration', 'modified_duration', 'ptr', 'residual_maturity', 'r_squared', 'roe', 'sharpe', 'sortino', 'std_dev', 'ter', 'tracking_error', 'treynor', 'ytm']
    metric_query = f"INSERT INTO transformed_metrics ({', '.join(metric_keys)}) VALUES ({', '.join(['%s'] * len(metric_keys))});"
    values = [amc_id, details["main_scheme_name"], mutual_fund_id] + [details.get("metrics", {}).get(col, '') for col in metric_keys[3:]]
    cursor.execute(metric_query, values)
    
conn.commit()
cursor.close()
conn.close()

print("Data inserted successfully!")


Data inserted successfully!


In [ ]:
for scheme_name, details in data.items():
    amc_id = get_or_create_amc(details["amc_name"])

    
    cursor.execute("""
        INSERT INTO mutual_funds (amc_id, benchmark_index, main_scheme_name, mutual_fund_name, monthly_aaum_date, monthly_aaum_value, scheme_launch_date, min_addl_amt, min_addl_amt_multiple, min_amt, min_amt_multiple) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        amc_id,  
        details["benchmark_index"],  
        details["main_scheme_name"],  
        details["mutual_fund_name"],  
        details["monthly_aaum_date"],  
        details["monthly_aaum_value"],  
        details["scheme_launch_date"],  
        details["min_addl_amt"],  
        details["min_addl_amt_multiple"],  
        details["min_amt"],  
        details["min_amt_multiple"]))

    mutual_fund_id = cursor.lastrowid

    #fund manager data
    # fund_manager_keys = ["amc_id", "mutual_fund_id", "main_scheme_name", "manager_name", "qualification", "managing_fund_since", "total_exp"]
    # fund_manager_query = f"INSERT INTO fund_managers ({', '.join(fund_manager_keys)}) VALUES ({', '.join(['%s'] * len(fund_manager_keys))});"

    # for manager in details.get("fund_manager", []):
    #     values = [amc_id, mutual_fund_id, details["main_scheme_name"]] + [manager.get(col, '') for col in fund_manager_keys[3:]]
    #     cursor.execute(fund_manager_query, values)

    
    for manager in details.get("fund_manager", []):
        cursor.execute("""
            INSERT INTO fund_managers (amc_id, mutual_fund_id, main_scheme_name, manager_name, qualification, managing_fund_since, total_exp) 
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (
            amc_id, 
            mutual_fund_id, 
            details["main_scheme_name"], 
            manager.get("name", ""), 
            manager.get("qualification", ""), 
            manager.get("managing_fund_since", ""), 
            manager.get("total_exp", "")))


    #load data
    # load_keys = ['amc_id', 'mutual_fund_id', 'main_scheme_name', 'entry_load', 'exit_load']
    # load_query = f"INSERT INTO loads ({', '.join(load_keys)}) VALUES ({', '.join(['%s'] * len(load_keys))});"
    # values = [amc_id,details["main_scheme_name"],mutual_fund_id] + [details.get("load",{}).get(col, '') for col in load_keys[3:]]
    # cursor.execute(load_query, values)
    
    entry_load = details.get("load",{}).get("entry_load","")
    exit_load =  details.get("load",{}).get("exit_load","")
    cursor.execute("""
        INSERT INTO loads (amc_id, mutual_fund_id, main_scheme_name, entry_load, exit_load)
        VALUES (%s ,%s, %s, %s, %s)
    """, (   
            amc_id,
            mutual_fund_id,
            details["main_scheme_name"],
            entry_load,  
            exit_load))
    
  
    #metrics
    metric_keys = ['amc_id','main_scheme_name','mutual_fund_id', 'alpha', 'avg_div_yld', 'avg_maturity', 'avg_pb', 'avg_pe', 'beta', 'jenson', 'macaulay_duration', 'modified_duration', 'ptr', 'residual_maturity', 'r_squared', 'roe', 'sharpe', 'sortino', 'std_dev', 'ter', 'tracking_error', 'treynor', 'ytm']
    metric_query = f"INSERT INTO metrics ({', '.join(metric_keys)}) VALUES ({', '.join(['%s'] * len(metric_keys))});"
    values = [amc_id,details["main_scheme_name"],mutual_fund_id] + [details.get("metrics",{}).get(col, '') for col in metric_keys[3:]]
    cursor.execute(metric_query, values)
    
    
conn.commit()
cursor.close()
conn.close()

print("Data inserted successfully!")

In [32]:
load_keys = ['amc_id', 'mutual_fund_id', 'main_scheme_name', 'entry_load', 'exit_load']

In [33]:
[details.get("load",{}).get(col, '') for col in load_keys[3:]]

['Not Applicable ',
 'Compulsory Lock-in Option 5 years or till the child attains age of majority (whichever is earlier). If redeemed before child attains 18 years of age, Exit load is 1 (Effective from May 3, 2018) Please refer to our Tata Mutual Fund website for fundamental changes, wherever applicable']

In [ ]:
# for load_type, load_value in details.get("load", {}).items():
#     cursor.execute("""
#         INSERT INTO loads (mutual_fund_id, load_type, load_value)
#         VALUES (%s, %s, %s)""", 
#         (mutual_fund_id, load_type, load_value))
    
# for metric_name, metric_value in details.get("metrics", {}).items():
#     cursor.execute("""
#         INSERT INTO metrics (mutual_fund_id, name, value)
#         VALUES (%s, %s, %s)""", 
#         (mutual_fund_id, metric_name, metric_value))


# min_amt = details.get("min_amt", {})
# if min_amt:
#     cursor.execute("""
#         INSERT INTO fund_minimum_amounts (mutual_fund_id, amt, thraftr)
#         VALUES (%s, %s, %s)""", 
#         (mutual_fund_id, min_amt.get("amt", "0"), min_amt.get("thraftr", "0")))

# min_addl_amt = details.get("min_addl_amt", {})
# if min_addl_amt:
#     cursor.execute("""
#         INSERT INTO fund_additional_amounts (mutual_fund_id, amt, thraftr)
#         VALUES (%s, %s, %s)""", 
#         (mutual_fund_id, min_addl_amt.get("amt", "0"), min_addl_amt.get("thraftr", "0")))

In [ ]:
 
# Axis Asset Management Company Limited
# Aditya Birla Sun Life AMC Limited
# Bank of India Investment Managers Private Limited
# Canara Robeco Asset Management Company Limited
# PGIM India Asset Management Private Limited
# DSP Investment Managers Private Limited
# Edelweiss Asset Management Limited
# Quant Money Managers Limited
# Franklin Templeton Asset Management (India) Private Limited
# HDFC Asset Management Company Limited
# HSBC Asset Management (India) Private Limited
# ICICI Prudential Asset Management Company Limited
# Bandhan AMC Limited
# 360 ONE Asset Management Limited
# IL&FS Infra Asset Management Limited
# Groww Asset Management Limited
# Invesco Asset Management (India) Private Limited
# JM Financial Asset Management Limited
# Kotak Mahindra Asset Management Company Limited(KMAMCL)
# LIC Mutual Fund Asset Management Limited
# Mahindra Manulife Investment Management Private Limited
# Mirae Asset Investment Managers (India) Private Limited
# Motilal Oswal Asset Management Company Limited
# PPFAS Asset Management Private Limited
# Quantum Asset Management Company Private Limited
# Nippon Life India Asset Management Limited
# SBI Funds Management Limited
# Shriram Asset Management Company Limited
# Sundaram Asset Management Company Limited
# Tata Asset Management Private Limited
# Taurus Asset Management Company Limited
# Union Asset Management Company Private Limited
# UTI Asset Management Company Limited
# WhiteOak Capital Asset Management Limited
# ITI Asset Management Limited
# Navi AMC Limited
# NJ Asset Management Private Limited
# Samco Asset Management Private Limited
# Trust Asset Management Private Limited
# Baroda BNP Paribas Asset Management India Private Limited
# Bajaj Finserv Asset Management Limited
# Helios Capital Asset Management (India) Private Limited
# Zerodha Asset Management Private Limited
# Old Bridge Asset Management Private Limited
# Angel One Asset Management Company Limited
# Unifi Asset Management Private Limited
 
 